# Data Model: `wx_data`

In [1]:
import os
from typing import Optional
from pathlib import Path
from datetime import datetime
from joblib import cpu_count, Parallel, delayed
import pandas as pd
from sqlmodel import SQLModel, Field, create_engine

# functions
def read_wxdata(FILE: Path) -> pd.DataFrame:
    """
    Read the wx_data.
    Input:
    - FILENAME: Path (from pathlib) object of the path of a wx_data file

    Output:
    - DF: dataframe of a wx_data
    """
    cols = ["date", "tmax", "tmin", "precip"]
    dtyp  = (str, int, int, int)
    dtypes  = {k:v for k, v in zip(cols, dtyp)}
    DF = pd.read_table(FILE,
                       delimiter = "\t",
                       header = None,
                       names  = cols, 
                       dtype  = dtypes)
    # Make time datetime obj
    DF["date"] = pd.to_datetime(DF["date"])
    # Assign a column for the site ID from the filename
    DF = DF.assign(site_id = FILE.stem)
    # Sanitize the null values
    DF = DF.replace(-9999, pd.NA)
    # print(df[df.isna().any(axis = 1)].head(3))
    return DF

In [2]:
dir_wxdata = Path("/code/wx_data/")
# Let's work on one file(
f_wxdata      = [f for f in dir_wxdata.glob("*")][0]
df = read_wxdata(f_wxdata)
print(df.head(3))

        date  tmax  tmin precip      site_id
0 1985-01-01   -22  -128     94  USC00110072
1 1985-01-02  -122  -217      0  USC00110072
2 1985-01-03  -106  -244      0  USC00110072


In [3]:
%%time
dfall = Parallel(n_jobs = cpu_count() - 1)(delayed(read_wxdata)(f) for f in dir_wxdata.glob("USC*.txt"))
dfall = pd.concat(dfall)

CPU times: user 598 ms, sys: 208 ms, total: 806 ms
Wall time: 3.46 s


## Create Data Model

In [4]:
# models.py
class WxTable(SQLModel, table = True):
    id: Optional[int] = Field(default = None, primary_key = True)
    site_id: str
    date:    datetime
    tmax:    int | None
    tmin:    int | None
    precip:  int | None

# app.py
# Create db engine
engine = create_engine(os.environ["DATABASE_URL"], echo = True)
SQLModel.metadata.create_all(engine)
# if we suffer a collation mismatch, it might due to previously using a different postgres and not deleting the volume.
# run the code below:
# > podman-compose down
# > podman volume ls # to find the name of the volume
# > podman volume rm <name-of-volume>
# > podman-compose up -d 

2026-01-19 02:52:15,276 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2026-01-19 02:52:15,278 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-19 02:52:15,281 INFO sqlalchemy.engine.Engine select current_schema()
2026-01-19 02:52:15,283 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-19 02:52:15,287 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2026-01-19 02:52:15,288 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-19 02:52:15,291 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-19 02:52:15,297 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [17]:
# Verify database
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.has_table("wxtable")

2026-01-19 02:54:58,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-19 02:54:58,390 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2026-01-19 02:54:58,394 INFO sqlalchemy.engine.Engine [cached since 163.1s ago] {'table_name': 'wxtable', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2026-01-19 02:54:58,398 INFO sqlalchemy.engine.Engine ROLLBACK


True

## Using scripts

In [ ]:
# SQLModel.metadata.clear()
# SQLModel.metadata.drop_all(engine)
main()

## Reading from database

In [1]:
%%time
import os
import pandas as pd
from main import main
from database import engine
main()

CPU times: user 2min 18s, sys: 1.08 s, total: 2min 19s
Wall time: 2min 37s


In [ ]:
pd.read_sql()

In [2]:
df = pd.read_sql("SELECT * FROM WxTable;", con = engine)
print("---")
print(df)

2026-01-19 19:09:20,299 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-19 19:09:20,302 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2026-01-19 19:09:20,306 INFO sqlalchemy.engine.Engine [cached since 22.53s ago] {'table_name': 'SELECT * FROM WxTable;', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2026-01-19 19:09:20,311 INFO sqlalchemy.engine.Engine SELECT * FROM WxTable;
2026-01-19 19:09:20,313 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-19 19:09:20,599 INFO sqlalchemy.engine.Engine ROLLBACK
---
    

# Create WxTable Statistics

In [5]:
from functions import read_wxdata
import pandas as pd
from pathlib import Path

df = read_wxdata(Path("/code/wx_data/USC00110338.txt"))


,date,tmax,tmin,precip,site_id
0,1985-01-01,11,-122,348,USC00110338
1,1985-01-02,-106,-183,3,USC00110338
2,1985-01-03,-56,-183,0,USC00110338
3,1985-01-04,-17,-161,0,USC00110338
4,1985-01-05,17,-61,0,USC00110338
...,...,...,...,...,...
10707,2014-12-27,106,-50,0,USC00110338
10708,2014-12-28,89,-56,0,USC00110338
10709,2014-12-29,17,-56,0,USC00110338
10710,2014-12-30,-6,-122,0,USC00110338


## Check for multiple records

In [38]:

for f in Path("/code/wx_data/").glob("USC*.txt"):
    df = read_wxdata(f)
    ind = df["date"].duplicated()
    if not df[ind].empty:
        print(df[ind])

## Calculating aggregate stats

In [39]:
import numpy as np
df.groupby(["site_id", df["date"].dt.year])\
  .agg(
      tmax_yearly   = ("tmax", "mean"),
      tmin_yearly   = ("tmin", "mean"),
      precip_yearly = ("precip", "sum") )\
  .reset_index()

,site_id,date,tmax_yearly,tmin_yearly,precip_yearly
0,USC00339312,1985,141.060274,39.550685,10067
1,USC00339312,1986,150.860274,51.315068,9476
2,USC00339312,1987,156.630137,52.794521,8115
3,USC00339312,1988,152.224044,36.622951,8982
4,USC00339312,1989,145.173077,39.808219,10183
5,USC00339312,1990,161.915068,55.720548,13540
6,USC00339312,1991,168.29589,57.487671,7417
7,USC00339312,1992,145.218579,44.680328,10898
8,USC00339312,1993,149.523288,45.810959,9987
9,USC00339312,1994,152.005479,40.169863,9471
